In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# kaggle https://www.kaggle.com/c/tmdb-box-office-prediction
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Checking to see the expected submission format
sample = pd.read_csv('../input/sample_submission.csv')

print(sample.head())

train = pd.read_csv('../input/train.csv')
print ("Train dataset has {} rows(samples) with {} columns(features) each.".format(*train.shape))
print(train.head())
#test = pd.read_csv('test.csv', index_col='Id')
test = pd.read_csv('../input/test.csv')
print(test.head())
print ("Test dataset has {} rows(samples) with {} columns(features) each.".format(*test.shape))

In [ ]:
X_train_prep=train[['budget','original_language','status','popularity','runtime']]
y_train=train['revenue']
X_train=pd.get_dummies(X_train_prep)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold,cross_val_score

def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'n_estimators': int(params['n_estimators']),
        'learning_rate': float(params['learning_rate']),
        'max_features': int(params['max_features']),
        'min_samples_leaf': int(params['min_samples_leaf']),
        'min_samples_split': int(params['min_samples_split']),
         
    }
    
    clf = GradientBoostingRegressor(**params, random_state = 0)
    
    print(params)
    score = cross_val_score(clf, X_train, y_train, scoring='neg_mean_squared_error', cv=StratifiedKFold(n_splits=5)).mean()
    print("rmse {:.3f} params {}".format(score, params))
    return score

space = {
    'max_depth': hp.quniform('max_depth', 2, 4,1),
    'n_estimators': hp.quniform('n_estimators', 20, 50,5),
    'learning_rate': hp.uniform('learning_rate', 0.05, 1.0),
    'max_features': hp.uniform('max_features', 2, 4),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.1, 5),
    'min_samples_split': hp.quniform('min_samples_split', 1, 2,1),
  
}

trials = Trials()
best = fmin(fn=objective, space=space,algo=tpe.suggest,   max_evals=10,trials=trials)
best_model = getBestModelfromTrials(trials)
print("Hyperopt estimated optimum {}".format(best))

In [ ]:
import sklearn
sklearn.metrics.SCORERS.keys()

In [ ]:
import xgboost as xgb
def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    }
    
    clf = xgb.XGBClassifier(
        n_estimators=250,
        learning_rate=0.05,
        n_jobs=4,
        **params
    )
    
    score = cross_val_score(clf, X_train_d, Y_train_d, scoring=gini_scorer, cv=StratifiedKFold()).mean()
    print("Gini {:.3f} params {}".format(score, params))
    return score

space = {
    'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
}

trials = Trials()
best = fmin(fn=objective, space=space,algo=tpe.suggest,   max_evals=10,trials=trials)
best_model = getBestModelfromTrials(trials)


In [ ]:
import lightgbm as lgb
def objective(params):
    params = {
        'num_leaves': int(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    }
    
    clf = lgbm.LGBMRegressor(
        n_estimators=500,
        learning_rate=0.01,
        **params
    )
    
    score = cross_val_score(clf, X, Y, scoring=gini_scorer, cv=StratifiedKFold()).mean()
    print("Gini {:.3f} params {}".format(score, params))
    return score

space = {
    'num_leaves': hp.quniform('num_leaves', 8, 128, 2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
}


trials = Trials()
best = fmin(fn=objective, space=space,algo=tpe.suggest,   max_evals=10,trials=trials)
best_model = getBestModelfromTrials(trials)
print("Hyperopt estimated optimum {}".format(best))

print("Hyperopt estimated optimum {}".format(best))

In [ ]:
def objective(params):
    params = {'n_estimators': int(params['n_estimators']), 'max_depth': int(params['max_depth'])}
    clf = RandomForestClassifier(n_jobs=4, class_weight='balanced', **params)
    score = cross_val_score(clf, X, Y, scoring=gini_scorer, cv=StratifiedKFold()).mean()
    print("Gini {:.3f} params {}".format(score, params))
    return score

space = {
    'n_estimators': hp.quniform('n_estimators', 25, 500, 25),
    'max_depth': hp.quniform('max_depth', 1, 10, 1)
}

trials = Trials()
best = fmin(fn=objective, space=space,algo=tpe.suggest,   max_evals=10,trials=trials)
best_model = getBestModelfromTrials(trials)
print("Hyperopt estimated optimum {}".format(best))

Gini -0.218 params {'n_estimators': 50, 'max_depth': 1}
Gini -0.243 params {'n_estimators': 175, 'max_depth': 4}
Gini -0.237 params {'n_estimat